In [4]:
import os
import re
import pickle
from typing import List
from pathlib import Path
from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [8]:
"""
Document Save/Load Utilities

List[Document]를 파일로 저장하고 로드하는 유틸리티 함수들
"""

def save_documents(docs: List[Document], output_path: str):
    """
    List[Document]를 pickle 파일로 저장합니다.
    
    Args:
        docs (List[Document]): 저장할 문서 리스트
        output_path (str): 저장할 디렉토리 경로
    """
    os.makedirs(output_path, exist_ok=True)
    docs_file_path = os.path.join(output_path, "documents.pkl")
    
    with open(docs_file_path, "wb") as f:
        pickle.dump(docs, f)
    
    print(f"✅ 문서 저장 완료: {docs_file_path}")
    print(f"📄 저장된 문서 수: {len(docs)}")


def load_documents(input_path: str) -> List[Document]:
    """
    저장된 pickle 파일에서 List[Document]를 로드합니다.
    
    Args:
        input_path (str): 문서가 저장된 디렉토리 경로 또는 파일 경로
        
    Returns:
        List[Document]: 로드된 문서 리스트
    """
    # 디렉토리 경로인 경우 documents.pkl 파일을 찾음
    if os.path.isdir(input_path):
        docs_file_path = os.path.join(input_path, "documents.pkl")
    else:
        docs_file_path = input_path
    
    if not os.path.exists(docs_file_path):
        raise FileNotFoundError(f"문서 파일을 찾을 수 없습니다: {docs_file_path}")
    
    with open(docs_file_path, "rb") as f:
        docs = pickle.load(f)
    
    print(f"✅ 문서 로드 완료: {docs_file_path}")
    print(f"📄 로드된 문서 수: {len(docs)}")
    
    return docs


def embed_and_save_with_docs(docs: List[Document], output_path: str, model_name: str = "bge-m3:latest"):
    """
    문서를 임베딩하고 FAISS 데이터베이스로 저장하며, 동시에 원본 문서도 저장합니다.
    
    Args:
        docs (List[Document]): 처리할 문서 리스트
        output_path (str): 저장할 디렉토리 경로
        model_name (str): 임베딩 모델명
    """
    # 임베딩 모델 초기화
    embedding_model = OllamaEmbeddings(model=model_name)
    
    # FAISS 데이터베이스 생성 및 저장
    db = FAISS.from_documents(docs, embedding_model)
    db.save_local(output_path)
    print(f"✅ 임베딩 저장 완료: {output_path}")
    
    # 원본 문서도 함께 저장
    save_documents(docs, output_path)
    

def load_both_faiss_and_docs(folder_path: str, model_name: str = "bge-m3:latest") -> tuple[FAISS, List[Document]]:
    """
    FAISS 벡터 데이터베이스와 원본 문서를 모두 로드합니다.
    
    Args:
        folder_path (str): 데이터가 저장된 디렉토리 경로
        model_name (str): 임베딩 모델명
        
    Returns:
        tuple: (FAISS 데이터베이스, List[Document])
    """
    # 임베딩 모델 초기화
    embedding_model = OllamaEmbeddings(model=model_name)
    
    # FAISS 데이터베이스 로드
    loaded_db = FAISS.load_local(
        folder_path=folder_path,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True,
    )
    print(f"✅ FAISS 데이터베이스 로드 완료: {folder_path}")
    
    # 원본 문서 로드
    docs = load_documents(folder_path)
    
    return loaded_db, docs


In [9]:
"""
Contextual Retrieval Class Implementation

이 모듈은 문서의 컨텍스트 정보를 생성하여 검색 품질을 향상시키는 
ContextualRetriever 클래스를 제공합니다.
"""

import os
from pathlib import Path
from typing import Optional, List
from langchain.docstore.document import Document
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


class ContextualRetriever:
    """Contextual Retrieval을 위한 클래스"""
    
    def __init__(self, whole_document: str, model_name: str = "exaone3.5:latest", temperature: float = 0.1):
        """
        ContextualRetriever 초기화
        
        Args:
            whole_document (str): 전체 문서 내용
            model_name (str): 사용할 LLM 모델명 (기본값: "exaone3.5:latest")
            temperature (float): LLM temperature 설정 (기본값: 0.1)
        """
        self.whole_document = whole_document
        self.model_name = model_name
        self.temperature = temperature
        
        # LLM 초기화
        self._initialize_llm()
        
        # 프롬프트 템플릿 설정
        self._setup_prompt()
        
        # 체인 구성
        self._setup_chain()
        
        print(f"🚀 ContextualRetriever 초기화 완료")
        print(f"📄 전체 문서 길이: {len(self.whole_document):,} characters")
        print(f"🤖 모델: {self.model_name}")
        print(f"🌡️ Temperature: {self.temperature}")
        
    def _initialize_llm(self):
        """LLM 초기화"""
        print(f"🤖 LLM 초기화 중... ({self.model_name})")
        self.llm = ChatOllama(
            model=self.model_name,
            temperature=self.temperature
        )
        
    def _setup_prompt(self):
        """프롬프트 템플릿 설정"""
        print("📝 프롬프트 템플릿 설정 중...")
        self.contextual_prompt = PromptTemplate.from_template(
            """<document> 
{WHOLE_DOCUMENT} 
</document> 
다음은 chunk 처리된 Document입니다. chunk의 내용은 전체 문서에서 일부분을 의미합니다.
<chunk> 
{CHUNK_CONTENT}
</chunk> 
- document의 맥락에서 chunk를 간단 명료하게 요약해주세요.
- 요약문의 최대 토큰은 150 이하여야 합니다. 
- 요약문의 목적은 Document 내에 재삽입하여 retriever를 통합 검색 품질을 높이기 위함입니다.
- 요약문은 한글로 작성해 주세요. 단, chunk내에 코드명, 영문 이니셜 혹은 영어 표현이 문서를 요약하는데 반드시 필요한 내용이라면 영어 그대로 포함될 수 있습니다.
- 요약문을 대표할 수 있는 용어(예, 영문 이니셜, 기능명 등)는 요약문 내에 포함해 주세요.
- 주요 코드값은 한국어로 변환하지 마세요.
- 요약문의 끝부분에 문서를 대표할 수 있는 키워드를 추가해주세요.
"""
        )
        
    def _setup_chain(self):
        """체인 구성"""
        print("⛓️ 체인 구성 중...")
        self.chain = self.contextual_prompt | self.llm | StrOutputParser()
        
    def get_contextual_text(self, chunk_content: str, section_path: str, verbose: bool = True) -> Optional[Document]:
        """
        주어진 청크에 대한 컨텍스트 정보를 생성하고 향상된 Document를 반환
        
        Args:
            chunk_content (str): 처리할 청크 내용
            verbose (bool): 상세 로그 출력 여부 (기본값: True)
            
        Returns:
            Document: 향상된 Document 객체 (성공시) 또는 None (실패시)
        """
        if verbose:
            print(f"\n🔍 컨텍스트 정보 생성 시작...")
            print(f"📊 청크 길이: {len(chunk_content):,} characters")
        
        try:
            # 컨텍스트 정보 생성
            context = self.chain.invoke({
                "WHOLE_DOCUMENT": self.whole_document,
                "CHUNK_CONTENT": chunk_content
            })
            
            if verbose:
                print("=" * 50)
                print(f"✅ 생성된 컨텍스트:\n{context}")
                print("=" * 50)
            
            # 향상된 Document 생성
            enhanced_content = f"[제목]: {section_path}\n\n[요약]: {context}\n\n[원문]: {chunk_content}"
            
            enhanced_doc = Document(
                page_content=enhanced_content,
                metadata={
                    "original_content": chunk_content,
                    "contextual_info": context,
                    "source": "contextual_retrieval",
                    "model": self.model_name,
                    "original_length": len(chunk_content),
                    "enhanced_length": len(enhanced_content)
                }
            )
            
            if verbose:
                print(f"✅ 향상된 Document 생성 완료!")
                print(f"📏 원본 길이: {len(chunk_content):,} characters")
                print(f"📏 향상된 길이: {len(enhanced_doc.page_content):,} characters")
                print(f"📈 증가율: {(len(enhanced_doc.page_content) / len(chunk_content) - 1) * 100:.1f}%")
            
            return enhanced_doc
            
        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            return None


    def get_stats(self) -> dict:
        """
        클래스 인스턴스의 통계 정보를 반환
        
        Returns:
            dict: 통계 정보 딕셔너리
        """
        return {
            "model_name": self.model_name,
            "temperature": self.temperature,
            "document_length": len(self.whole_document),
            "document_size_mb": len(self.whole_document) / (1024 * 1024),
        }
    
    def __repr__(self) -> str:
        """클래스의 문자열 표현"""
        return f"ContextualRetriever(model='{self.model_name}', doc_size={len(self.whole_document):,} chars)"




In [10]:
def load_markdown_file(file_path: str) -> str:
    """마크다운 파일을 로드합니다."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

contextual_retriever = ContextualRetriever(
    whole_document=load_markdown_file("data/dev_center_guide_allmd_touched.md"),
    model_name="exaone3.5:latest",
    temperature=0.1
)

print("\n" + "="*60)
print("✅ 클래스 초기화 완료 (import된 클래스 사용)!")
print("="*60)


🤖 LLM 초기화 중... (exaone3.5:latest)
📝 프롬프트 템플릿 설정 중...
⛓️ 체인 구성 중...
🚀 ContextualRetriever 초기화 완료
📄 전체 문서 길이: 383,563 characters
🤖 모델: exaone3.5:latest
🌡️ Temperature: 0.1

✅ 클래스 초기화 완료 (import된 클래스 사용)!


In [ ]:
def hierarchical_markdown_split(md_text: str, path_prefix: str = "") -> list[Document]:
    """마크다운 문서를 계층적으로 분할합니다."""
    splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
        ("#", "title"),
        ("##", "section"),
        ("###", "subsection"),
        ("####", "subsubsection"),
        ("#####", "subsubsubsection")
    ])
    docs = splitter.split_text(md_text)

    result_docs = []
    current_title = None
    chunk_idx = 0
    for doc in docs:
        metadata = doc.metadata
        if "title" in metadata:
            current_title = metadata["title"]

        if current_title:
            chunk_idx += 1
            full_title = "" + current_title
            if "section" in metadata:
                full_title += f" / {metadata['section']}"
            if "subsection" in metadata:
                full_title += f" / {metadata['subsection']}"
            if "subsubsection" in metadata:
                full_title += f" / {metadata['subsubsection']}"
            if "subsubsubsection" in metadata:
                full_title += f" / {metadata['subsubsubsection']}"

            # content = f"[section_path]: {full_title}\n\n{doc.page_content}"
            # doc = Document(page_content=content, metadata={
            #     **doc.metadata,
            #     "type": "documentation",
            #     "source": "dev_center_guide_allmd.md",
            #     "chunk_idx": chunk_idx
            # })
            doc = contextual_retriever.get_contextual_text(doc.page_content, section_path=full_title)

        result_docs.append(doc)

    return result_docs

def load_markdown_file(file_path: str) -> str:
    """마크다운 파일을 로드합니다."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# 마크다운 파일 로드 및 분할
# allmd 적용시 검색 품질의 급격한 저하 확(495 chunks)
str_md_file = load_markdown_file("data/dev_center_guide_allmd_touched.md") 
docs_markdown = hierarchical_markdown_split(str_md_file)

print(f"마크다운 문서 분할 완료: {len(docs_markdown)}개 청크")




🔍 컨텍스트 정보 생성 시작...
📊 청크 길이: 1,165 characters
✅ 생성된 컨텍스트:
**요약:**

원스토어 인앱결제 API V7(SDK V21)은 최신 기능과 강력한 보안을 제공하며, 이전 버전(V4 이하)과 호환되지 않습니다. 주요 기능은 결제 테스트, 서버 API 활용, 정기 결제 구현 등이 포함됩니다. 개발자는 이 가이드를 통해 사전 준비부터 구현, 테스트, 보안까지 전반적인 인앱결제 프로세스를 이해하고 적용할 수 있습니다. **핵심 용어:** 원스토어 인앱결제 API V7, SDK V21, 결제 구현, 정기 결제  
**키워드:** 인앱결제, API V7, SDK 구현, 정기결제
✅ 향상된 Document 생성 완료!
📏 원본 길이: 1,165 characters
📏 향상된 길이: 1,510 characters
📈 증가율: 29.6%

🔍 컨텍스트 정보 생성 시작...
📊 청크 길이: 307 characters
✅ 생성된 컨텍스트:
**요약:**

원스토어 IAP 서비스는 안드로이드 앱 내 상품 판매 및 결제를 위한 인증 및 결제 시스템을 제공합니다. 개발자는 '원스토어 IAP SDK'를 활용해 앱 내 결제 기능을 구현하고, 이를 통해 사용자 결제를 처리하고 정산받을 수 있습니다. 주요 용어: **IAP SDK**, **결제 서버**  
**키워드:** IAP, 결제 연동, SDK 적용
✅ 향상된 Document 생성 완료!
📏 원본 길이: 307 characters
📏 향상된 길이: 599 characters
📈 증가율: 95.1%

🔍 컨텍스트 정보 생성 시작...
📊 청크 길이: 894 characters
✅ 생성된 컨텍스트:
**요약:**

원스토어 인앱결제 API V7(SDK V21)는 관리형 상품과 구독형 상품을 지원합니다. 관리형 상품은 일회성 구매로 소비 전까지 재구매가 필요하며, 구독형 상품은 정기 결제를 원스토어에서 처리하여 사용자 편의성을 높입니다. 구독형 상품은 프로모션 기능과 결제 수단 변경 등 다양한 혜택을

In [20]:
# for doc in docs_markdown[:10]:
#     print(doc.page_content)
#     print("-" * 100)
#     print(doc.metadata.get("model", "Unknown"))
#     print("-" * 100)
# save_documents(docs_markdown, "models/temp_docs")

docs = load_documents("models/temp_docs")
# print("-- lens -->" + len(docs))

for doc in docs:
    print(doc.page_content)
    print("-" * 100)
    print(doc.metadata.get("model", "Unknown"))
    print("-" * 100)

✅ 문서 로드 완료: models/temp_docs/documents.pkl
📄 로드된 문서 수: 10
출처: https://onestore-dev.gitbook.io/dev/tools/billing/v21.md
----------------------------------------------------------------------------------------------------
Unknown
----------------------------------------------------------------------------------------------------
[제목]: 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드

[요약]: **요약:**

원스토어 인앱결제 API V7(SDK V21)은 최신 기능과 강력한 보안을 제공하며, 이전 버전(V4 이하)과 호환되지 않습니다. 주요 기능은 결제 테스트, 서버 API 활용, 정기 결제 구현 등이 포함됩니다. 개발자는 이 가이드를 통해 사전 준비부터 구현, 테스트, 보안까지 전반적인 인앱결제 프로세스를 이해하고 적용할 수 있습니다. **핵심 용어:** 원스토어 인앱결제 API V7, SDK V21, 결제 구현, 정기 결제  
**키워드:** 인앱결제, API V7, SDK 구현, 정기결제

[원문]: 원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.  
보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.  
{% hint style="info" %}
API V4(SDK V16) 이하 버전과는 호환되지 않습니다. 인앱결제 API V4(SDK V16)에 대한 안내 및 다운로드는 [여기](old-version/v16)를 클릭해주세요.
{% endhint %}  
{% hint style="info" %}
현재 판매중인 앱을 대한민국 외 국가/지역으로 배포하기 위해서는 아래 가이드를 참고해주세요  
* [대한민국 외 국가 및 지역 배포를 위한 가이

In [12]:
# fixed_model_name = "deepseek-coder:6.7b"
# fixed_model_name = "exaone3.5:latest"
# fixed_model_name = "mistral:latest"
# fixed_model_name = "llama3:8b"
fixed_model_name = "bge-m3:latest"    
# fixed_model_name = "solar:latest"

def embed_and_save(docs: List[Document], output_path: str):
    """문서를 임베딩하고 FAISS 데이터베이스로 저장합니다."""
    # 임베딩 모델 초기화
    embedding_model = OllamaEmbeddings(model=fixed_model_name)
    
    # FAISS 데이터베이스 생성 및 저장
    db = FAISS.from_documents(docs, embedding_model)
    db.save_local(output_path)
    print(f"✅ 임베딩 저장 완료: {output_path}")

# 모든 문서 통합
total_docs = docs_markdown #+ docs_kotlin
print(f"총 {len(total_docs)}개의 문서를 생성하였습니다.")
print(f"- 마크다운 문서: {len(docs_markdown)}개")
#print(f"- Kotlin 코드: {len(docs_kotlin)}개")

# 임베딩 생성 및 저장
output_dir = "models/faiss_vs_rag_iap_v11_cr2_" + fixed_model_name[:3]
os.makedirs(output_dir, exist_ok=True)
# embed_and_save(total_docs, output_dir)
embed_and_save_with_docs(total_docs, output_dir)

총 495개의 문서를 생성하였습니다.
- 마크다운 문서: 495개
✅ 임베딩 저장 완료: models/faiss_vs_rag_iap_v11_cr2_bge
✅ 문서 저장 완료: models/faiss_vs_rag_iap_v11_cr2_bge/documents.pkl
📄 저장된 문서 수: 495


In [21]:
# models/faiss_vs_rag_iap_v11_cr2_bge/documents.pkl
docs = load_documents("models/faiss_vs_rag_iap_v11_cr2_bge")

for doc in docs[:10]:
    print(doc.metadata.get("model", "Unknown"))
    print(doc.page_content)
    print("-" * 100)

✅ 문서 로드 완료: models/faiss_vs_rag_iap_v11_cr2_bge/documents.pkl
📄 로드된 문서 수: 495
Unknown
출처: https://onestore-dev.gitbook.io/dev/tools/billing/v21.md
----------------------------------------------------------------------------------------------------
exaone3.5:latest
[제목]: 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드

[요약]: **요약:**

원스토어 인앱결제 API V7(SDK V21)은 최신 기능과 강력한 보안을 제공하며, 이전 버전(V4 이하)과 호환되지 않습니다. 주요 기능은 결제 테스트, 서버 API 활용, 정기 결제 구현 등이 포함됩니다. 이 버전은 앱 배포 시 대한민국 외 지역 가이드도 함께 제공됩니다. **핵심 용어:** 원스토어 인앱결제 V7, SDK V21, 결제 구현, 정기 결제  
**키워드:** 인앱결제, SDK 업그레이드, 결제 기능

[원문]: 원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.  
보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.  
{% hint style="info" %}
API V4(SDK V16) 이하 버전과는 호환되지 않습니다. 인앱결제 API V4(SDK V16)에 대한 안내 및 다운로드는 [여기](old-version/v16)를 클릭해주세요.
{% endhint %}  
{% hint style="info" %}
현재 판매중인 앱을 대한민국 외 국가/지역으로 배포하기 위해서는 아래 가이드를 참고해주세요  
* [대한민국 외 국가 및 지역 배포를 위한 가이드](../glb)
{% endhint %}  
If you are comfortable with English, please change the language to English fr

In [57]:
# docs_markdown에서 'launchPurchaseFlow()' 문자열을 포함하는 Documnet 를 출력

cnt = 0
for doc in docs_markdown:
    if 'PNS' in doc.page_content:
        # print(f"문서 제목: {doc.metadata.get('title', '제목 없음')}")
        print(f"--- doc_index: {cnt} ---")
        print(f"{doc.page_content}[EOD]")  # 처음 200자만 출력
        # print("-" * 40)  # 구분선 
        cnt += 1
        
print(f"'PNS' 문자열을 포함하는 문서 개수: {cnt}")
        
        

--- doc_index: 0 ---
[제목]: 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드

[요약]: **요약:**

원스토어 인앱결제 API V7(SDK V21)은 최신 기능과 강력한 보안을 제공하며, 이전 버전(V4 이하)과 호환되지 않습니다. 주요 기능은 결제 테스트, 서버 API 활용, 정기 결제 구현 등이 포함됩니다. 이 버전은 앱 배포 시 대한민국 외 지역 가이드도 함께 제공됩니다. **API V7**, **결제 구현**, **서버 API**, **앱 배포 가이드**

[원문]: 원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.  
보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.  
{% hint style="info" %}
API V4(SDK V16) 이하 버전과는 호환되지 않습니다. 인앱결제 API V4(SDK V16)에 대한 안내 및 다운로드는 [여기](old-version/v16)를 클릭해주세요.
{% endhint %}  
{% hint style="info" %}
현재 판매중인 앱을 대한민국 외 국가/지역으로 배포하기 위해서는 아래 가이드를 참고해주세요  
* [대한민국 외 국가 및 지역 배포를 위한 가이드](../glb)
{% endhint %}  
If you are comfortable with English, please change the language to English from the upper left side in this page.  
* [01. 원스토어 인앱결제 개요](v21/ov)
* [02. 원스토어 인앱결제 적용을 위한 사전준비](v21/pre)
* [03. 결제 테스트 및 보안](v21/test)
* [04. 원스토어 인앱결제 SDK를 사용해 구현하기](v21/sdk)
* [05. 원스토어 인앱결제 레퍼런스](v21/references)
* [06. 원스토어 인앱결제 서버 API (API V7)](v21/serverapi)
* [07. PNS(

In [23]:
###### Retriever Check from FAISS Vector DB ######

from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import BM25Retriever

docs_markdown = load_documents("models/faiss_vs_rag_iap_v11_cr2_bge")
fixed_model_name = "bge-m3:latest"

# ✅ 3. 임베딩 모델 초기화 (Ollama)
embedding_model = OllamaEmbeddings(model=fixed_model_name)

# 저장된 데이터를 로드
loaded_db = FAISS.load_local(
    folder_path=output_dir,
    # index_name="index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
)

retriever = loaded_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 30, "fetch_k": 70, "lambda_mult": 0.7}
    # search_type="similarity",
    # search_kwargs={"k": 50}
)

bm25 = BM25Retriever.from_documents(
    docs_markdown,
    bm25_params={"k1": 1.5, "b": 0.75}
)

bm25.k = 30

ensembled_retriever = EnsembleRetriever(
    retrievers=[bm25, retriever],
    weights=[0.5, 0.5]
)


res = ensembled_retriever.invoke(
    # "원스토어 인앱결제의 PNS의 개념을 설명해주세요"
    "PNS 메시지 규격의 purchaseState는 어떤 값으로 구성되나요?"
)

# res = bm25.invoke(
#     # "PNS의 메세지 규격을 설명해주세요",
#     "PNS 메시지 규격의 purcahseState는 어떤 값으로 구성되나요?"
# )

# res = bm25.invoke(
#     # "원스토어 인앱결제의 PNS의 개념을 설명해주세요",
#     "PNS 메시지 규격의 purcahseState는 어떤 값으로 구성되나요?"
# )

print(f"검색된 문서 수: {len(res)}")

idx = 0
for doc in res: 
    print(f"--- doc_index: {idx} ---")
    print(doc.page_content)  # Print first 100 characters of each document
    # print(doc.metadata)
    # print('-' * 40)
    idx += 1

✅ 문서 로드 완료: models/faiss_vs_rag_iap_v11_cr2_bge/documents.pkl
📄 로드된 문서 수: 495
검색된 문서 수: 53
--- doc_index: 0 ---
[제목]: 09. 원스토어 인앱결제 릴리즈 노트 / **원스토어 인앱결제 라이브러리 API V6(SDK V19) 출시** <a href="#id-09.-apiv6-sdkv19" id="id-09.-apiv6-sdkv19"></a> / PNS 메시지 규격 변경  <a href="#id-09.-pns" id="id-09.-pns"></a>

[요약]: **요약:**

결제 API 문서는 원화 외 통화 지원을 위해 결제금액과 결제수단 금액의 데이터 타입을 String으로 변경하고, 통화코드(priceCurrencyCode)를 추가했습니다. 응답 규격에는 purchaseToken, environment, marketCode 필드가 추가되었습니다. 주요 기능은 주문 정보 조회, 결제 처리, 결제 결과 확인으로 구성되어 있으며, 다양한 통화 및 환경 지원을 통해 글로벌 결제 시스템을 강화하고 있습니다. **결제API, 글로벌결제, 통화지원**

[원문]: * 원화 외의 통화를 지원하기 위하여 결제금액(price)의 데이터 타입이 Number에서 String으로 변경하였습니다.
* 원화 외의 통화를 지원하기 위하여 결제금액의 통화코드(priceCurrencyCode)를 추가하였습니다.
* 원화 외의 통화를 지원하기 위하여 결제수단별금액(amount)의 데이터 타입이 Number에서 String으로 변경하였습니다.
* 응답 규격에 purchaseToken, environment, marketCode 필드가 추가되었습니다.&#x20;  
상세한 규격은 PNS 메시지 상세 변경 내역에서 확인 할 수 있습니다. &#x20;
--- doc_index: 1 ---
[제목]: 07. PNS(Payment Notification Service) 이용하기 / Notification 전송 정책